#Load Data 


In [41]:
import pandas as pd
import numpy as np


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
 ! pip install -q kaggle

In [6]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/CAPSTONE/Husein/Collaborative_filterring_learn"

In [7]:
%cd /content/drive/MyDrive/CAPSTONE/Husein/Collaborative_filterring_learn


/content/drive/MyDrive/CAPSTONE/Husein/Collaborative_filterring_learn


In [8]:
!kaggle datasets download -d rounakbanik/the-movies-dataset

the-movies-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [9]:
!ls -a

Collaborative_filterring_learn.ipynb  movies_metadata.csv
credits.csv			      ratings.csv
kaggle.json			      ratings_small.csv
keywords.csv			      Recomendation_based_content.ipynb
links.csv			      Simpler_Recomender.ipynb
links_small.csv			      the-movies-dataset.zip
model.png


##Build 1

We will build a system that recommends movies that are similar to a particular movie. To achieve this, you will compute pairwise cosine similarity scores for all movies based on their descriptions and recommend movies based on that similarity score threshold.

The description is available to you as the overview feature in your metadata dataset

In [10]:

df_moviemetadata= pd.read_csv('/content/drive/MyDrive/CAPSTONE/Husein/Collaborative_filterring_learn/movies_metadata.csv')


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
df_moviemetadata.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [12]:
df_moviemetadata.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [13]:
df_moviemetadata.isnull().sum() # dari data tersebut, kolom overview akan dijadikan pembanding. Oleh sebab itu data tidak boleh kosong

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

In [14]:
df_moviemetadata['overview']=df_moviemetadata['overview'].fillna('') #lalu kita build data terisi pada bagian kolom yang kosong
print(df_moviemetadata['overview'].isnull().sum() ) #disini data kolom overview sudah 0

0


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words= 'english')
tfidf_matrix = tfidf.fit_transform(df_moviemetadata['overview'])
tfidf_matrix.shape

(45466, 75827)

`We see that over 75827 different words were used to describe the 45466 movies in our dataset.`

In [16]:
tfidf.get_feature_names()[1:10]

['000', '000km', '000th', '001', '006', '007', '008', '009', '0093']

In [17]:
tfidf.get_feature_names()[7000:7010]

['berthe',
 'berthelsen',
 'berthier',
 'berthold',
 'berti',
 'bertie',
 'bertier',
 'bertignac',
 'bertil',
 'bertinelli']

`Dengan matriks ini di tangan, Anda sekarang dapat menghitung skor kesamaan. Ada beberapa metrik kemiripan yang dapat Anda gunakan untuk ini, seperti skor kemiripan manhattan, euclidean, Pearson, dan cosinus. Sekali lagi, tidak ada jawaban yang tepat untuk skor mana yang terbaik.

Anda akan menggunakan kesamaan kosinus untuk menghitung kuantitas numerik yang menunjukkan kesamaan antara dua film. Anda menggunakan skor kesamaan kosinus karena tidak bergantung pada besarnya dan relatif mudah dan cepat untuk dihitung.

Karena kita telah menggunakan vektorizer TF-IDF, menghitung perkalian titik antara setiap vektor akan langsung memberi Anda skor kesamaan kosinus. Oleh karena itu, Anda akan menggunakan linear_kernel () sklearn daripada cosine_similarities () karena lebih cepat.`


`cos(θ)=x.y||x||||y||`
 
So we can use the linear_kernel from sklearn, which is faster than cosine_similarity

In [18]:
from sklearn.metrics.pairwise import linear_kernel
#bikin fungsi
tfidf_matrix=tfidf_matrix[:30000] #batasi data sampai 30000 agar memory waktu preprocessing tidak terkendala apapun dan memori cukup


#bikin cosinus perhitungannya
cosine_sim=linear_kernel(tfidf_matrix,tfidf_matrix)

In [19]:
print(cosine_sim.shape)

(30000, 30000)


In [20]:
print(cosine_sim[1])

[0.01504121 1.         0.04681953 ... 0.         0.         0.02303661]


`We need to map movie titles and DataFrame indices. In other words, you need a mechanism to identify the index of a movie in your metadata DataFrame, given its title : Kita perlu memetakan judul film dan indeks DataFrame. Dengan kata lain, Anda memerlukan mekanisme untuk mengidentifikasi indeks film di metadata Anda, DataFrame, dengan judulnya..`

In [21]:
#bikin variable baru indices dimana, kolom tittles sebagai map 
indices = pd.Series(index=df_moviemetadata['title'],data=df_moviemetadata.index).drop_duplicates()
print(indices)

title
Toy Story                          0
Jumanji                            1
Grumpier Old Men                   2
Waiting to Exhale                  3
Father of the Bride Part II        4
                               ...  
Subdue                         45461
Century of Birthing            45462
Betrayal                       45463
Satan Triumphant               45464
Queerama                       45465
Length: 45466, dtype: int64


In [22]:
# make a function
def get_recomendations(title ,cosine_sim=cosine_sim):
  idx=indices[title] # cari index film yang sesuai dengan judul
  sim_scores=list(enumerate(cosine_sim[idx])) # bikn skor kesamaan antar2 film 
  sim_scores=sorted(sim_scores, key=lambda x:x[1],reverse=True) #film akan diurutkan berdasarkan kemiripan kesamaan
  sim_scores=sim_scores[1:11] #score 10 film yang hampir sama
  movie_indices = [i[0] for i in sim_scores] # # Get the movie indices
  return df_moviemetadata['title'][movie_indices]


In [23]:
get_recomendations('The Dark Knight Rises')

12481                                      The Dark Knight
150                                         Batman Forever
1328                                        Batman Returns
15511                           Batman: Under the Red Hood
585                                                 Batman
21194    Batman Unmasked: The Psychology of the Dark Kn...
9230                    Batman Beyond: Return of the Joker
18035                                     Batman: Year One
19792              Batman: The Dark Knight Returns, Part 1
3095                          Batman: Mask of the Phantasm
Name: title, dtype: object

In [24]:
df_moviemetadata.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


## Build2 : Adding Feauture

We will build a recommender system based on the following metadata: the 3 top actors, the director, related genres, and the movie keywords.

In [25]:
import pandas as pd

In [26]:
df_credits= pd.read_csv('/content/drive/MyDrive/CAPSTONE/Husein/Collaborative_filterring_learn/credits.csv') # didalam credits ada nama aktor yang merujuk index
df_keywords=pd.read_csv('/content/drive/MyDrive/CAPSTONE/Husein/Collaborative_filterring_learn/keywords.csv')

In [27]:
df_moviemetadata = df_moviemetadata.drop([19730, 29503, 35587])


In [28]:
# Convert IDs to int. Required for merging
df_keywords['id'] = df_keywords['id'].astype('int')
df_credits['id'] = df_credits['id'].astype('int')
df_moviemetadata['id'] = df_moviemetadata['id'].astype('int')

# Merge keywords and credits into your main metadata dataframe
df_moviemetadata = df_moviemetadata.merge(df_credits, on='id')
df_moviemetadata = df_moviemetadata.merge(df_keywords, on='id')
df_moviemetadata.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."


`Merge file credits and keyword berdasarkan 'id' kedalam data utama(movie)`

In [29]:
df_moviemetadata.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'cast', 'crew', 'keywords'],
      dtype='object')

In [30]:
df_moviemetadata=df_moviemetadata[:2000]
features = ['cast', 'crew', 'keywords', 'genres']
df_moviemetadata[features].head(2)

,cast,crew,keywords,genres
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...","[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1...","[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."


In [31]:
# Parse the striliteral_evalfied features into their corresponding python objects
from ast import literal_eval

for feature in features:
    df_moviemetadata[feature] = df_moviemetadata[feature].apply(literal_eval)

df_moviemetadata[features].head(2) 

,cast,crew,keywords,genres
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...","[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1...","[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."


In [33]:
df_moviemetadata['genres'][0]

[{'id': 16, 'name': 'Animation'},
 {'id': 35, 'name': 'Comedy'},
 {'id': 10751, 'name': 'Family'}]

In [34]:
df_moviemetadata['keywords'][0]

[{'id': 931, 'name': 'jealousy'},
 {'id': 4290, 'name': 'toy'},
 {'id': 5202, 'name': 'boy'},
 {'id': 6054, 'name': 'friendship'},
 {'id': 9713, 'name': 'friends'},
 {'id': 9823, 'name': 'rivalry'},
 {'id': 165503, 'name': 'boy next door'},
 {'id': 170722, 'name': 'new toy'},
 {'id': 187065, 'name': 'toy comes to life'}]

In [37]:
#df_moviemetadata['crew'][0]

Get the director's name from the crew feature. If the director is not listed, return NaN

In [38]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

Next, you will write a function that will return the top 3 elements or the entire list, whichever is more. Here the list refers to the cast, keywords, and genres.

The isinstance() function returns True if the specified object is of the specified type, otherwise False.

In [39]:
def get_list_or_top_3(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        # get just 3 or less
        if len(names) > 3:
            names = names[:3]
        return names

    # return empty list if it missing data!
    return []

In [42]:
# Define new director
df_moviemetadata['director'] = df_moviemetadata['crew'].apply(get_director)
df_moviemetadata.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords,director
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...",John Lasseter
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1...",Joe Johnston


In [44]:
# Define new director cast, genres and keywords features.
features = ['cast', 'keywords', 'genres']
for feature in features:
    df_moviemetadata[feature] = df_moviemetadata[feature].apply(get_list_or_top_3)
    
df_moviemetadata[['title', 'cast', 'director', 'keywords', 'genres']].head(3)

,title,cast,director,keywords,genres
0,Toy Story,"[Tom Hanks, Tim Allen, Don Rickles]",John Lasseter,"[jealousy, toy, boy]","[Animation, Comedy, Family]"
1,Jumanji,"[Robin Williams, Jonathan Hyde, Kirsten Dunst]",Joe Johnston,"[board game, disappearance, based on children'...","[Adventure, Fantasy, Family]"
2,Grumpier Old Men,"[Walter Matthau, Jack Lemmon, Ann-Margret]",Howard Deutch,"[fishing, best friend, duringcreditsstinger]","[Romance, Comedy]"
